In [1]:
from py2neo import Graph, Node, Relationship, Path, LIKE
from py2neo.matching import *
import pandas as pd

In [2]:
!pip install neo4jupyter
!pip install vis
import neo4jupyter
neo4jupyter.init_notebook_mode()

You should consider upgrading via the 'c:\users\mohammad\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mohammad\python37\python.exe -m pip install --upgrade pip' command.


<IPython.core.display.Javascript object>

In [4]:
data = [
    ["John", "Works","Apple"],
    ["John", "knows","python"],
    ["John", "knows","SQL"],
    ["Mary", "Works","IBM"],
    ["Mary", "knows","python"],
    ["Mary", "knows","R"],
    ["Sarah", "knows","python"],
    ["IBM", "requires","R"],
    ["IBM", "requires","python"],
    ["Apple", "requires","SQL"],
    ["Apple", "requires","R"],
]


data = pd.DataFrame(data, columns = ['node1',"edge","node2"])
data

,node1,edge,node2
0,John,Works,Apple
1,John,knows,python
2,John,knows,SQL
3,Mary,Works,IBM
4,Mary,knows,python
5,Mary,knows,R
6,Sarah,knows,python
7,IBM,requires,R
8,IBM,requires,python
9,Apple,requires,SQL


In [5]:
graph = Graph("bolt://localhost:7687", password="comemit", user = 'neo4j')
#graph = Graph("http://localhost:7474", password="comemit", user = 'neo4j')

graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [6]:
graph.call.dbms.components()

 name         | versions  | edition    
--------------|-----------|------------
 Neo4j Kernel | ['4.2.1'] | enterprise 

In [7]:
def create_db(node1, node2, edge):
  
    if edge == 'Works':
        
        a = Node("Person", name=node1)
        a.__primarykey__ = "name"
        a.__primarylabel__ = "Person"
        graph.merge(a)
        
        b = Node("Company", name=node2)
        b.__primarykey__ = "name"
        b.__primarylabel__ = "Company"
        graph.merge(b)
        
        graph.merge(Relationship(a, "WORKS", b))
        
    if edge == 'knows': 
        
        a = Node("Person", name=node1)
        a.__primarykey__ = "name"
        a.__primarylabel__ = "Person"
        graph.merge(a)
        
        b = Node("Skill", name=node2)
        b.__primarykey__ = "name"
        b.__primarylabel__ = "Skill"
        graph.merge(b)
        
        graph.merge(Relationship(a, "KNOWS", b))
        
    if edge == 'requires':
        
        a = Node("Company", name=node1)
        a.__primarykey__ = "name"
        a.__primarylabel__ = "Company"
        graph.merge(a)
        
        b = Node("Skill", name=node2)
        b.__primarykey__ = "name"
        b.__primarylabel__ = "Skill"
        graph.merge(b)
        
        graph.merge(Relationship(a, "REQUIRES", b))
        
        
        
list(map(create_db, data['node1'], data['node2'],data['edge']));

        

In [9]:
options = {"Person": "name", "Skill": "name","Company":"name"}
neo4jupyter.draw(graph, options = options, physics=True )

Explore the graph

In [28]:
graph.run('''MATCH (n:Person)
    RETURN count(n) as count
    ''')

 count 
-------
     3 

In [10]:
nodes = NodeMatcher(graph)
nodes.match("Person").\
where(name=STARTS_WITH("J")).\
all()

[Node('Person', name='John')]

In [11]:
relationships=RelationshipMatcher (graph)

ibm_node = graph.nodes.match("Company", name="Keanu IBM")

relationships.match((None, None),"REQUIRES").all()        #(None means any node);

[REQUIRES(Node('Company', name='IBM'), Node('Skill', name='python')),
 REQUIRES(Node('Company', name='IBM'), Node('Skill', name='R')),
 REQUIRES(Node('Company', name='Apple'), Node('Skill', name='R')),
 REQUIRES(Node('Company', name='Apple'), Node('Skill', name='SQL'))]

Directly run cypher queries

In [31]:
df=graph.run('''
          MATCH (b:Skill {name: "python"})<-[:KNOWS]-(a:Person)-[:WORKS]->(c:Company)
          RETURN a.name, b.name, c.name
          ORDER BY  a.name DESC
          ''').to_data_frame()

df

,a.name,b.name,c.name
0,Mary,python,IBM
1,John,python,Apple


Extract a subgraph

In [24]:
r=graph.run('''
          MATCH (a:Person)-[r:WORKS]->(c:Company {name:'IBM'})
          RETURN r
          ''').to_subgraph()
r

WORKS(Node('Person', name='Mary'), Node('Company', name='IBM'))

In [32]:
options = {"Person": "name", "Skill": "name","Company":"name"}
neo4jupyter.draw_subgraph(r, options = options, physics=True )